In [1]:
import sqlite3
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

conn = sqlite3.connect('example.db')
def rsq(sql):
    df = pd.read_sql(sql, conn)
    return df

In [2]:
rsq("""-- Select the average of home + away goals, multiplied by 3
SELECT 
    3 * avg(home_goal + away_goal)
FROM match;""")

,3 * avg(home_goal + away_goal)
0,8.575419


In [3]:
rsq("""SELECT 
    -- Select the date, home goals, and away goals scored
    date,
    home_goal,
    away_goal
FROM  match
-- Filter for matches where total goals exceeds 3x the average
WHERE (home_goal + away_goal) > 
       (SELECT 3 * AVG(home_goal + away_goal)
        FROM match); """)

,date,home_goal,away_goal
0,2013-01-18,5,4.0
1,2015-02-14,4,5.0
2,2014-10-25,4,5.0
3,2011-08-28,8,2.0
4,2012-12-29,7,3.0
5,2013-05-19,5,5.0
6,2013-03-30,9,2.0
7,2011-11-06,6,4.0
8,2013-10-30,7,3.0
9,2015-04-05,9,1.0


In [4]:
rsq("""SELECT 
    -- Select the team long and short names
    team_long_name,
    team_short_name
FROM team
-- Exclude all values from the subquery
WHERE team_api_id NOT IN
     (SELECT DISTINCT hometeam_id FROM match);""")

,team_long_name,team_short_name
0,KRC Genk,GEN
1,Beerschot AC,BAC
2,SV Zulte-Waregem,ZUL
3,Sporting Lokeren,LOK
4,KSV Cercle Brugge,CEB
...,...,...
223,FC St. Gallen,GAL
224,FC Thun,THU
225,Servette FC,SER
226,FC Lausanne-Sports,LAU


In [5]:
rsq("""SELECT
    -- Select the team long and short names
    team_long_name,
    team_short_name
FROM team
-- Filter for teams with 8 or more home goals
WHERE team_api_id IN
      (SELECT hometeam_id 
       FROM match
       WHERE home_goal >= 8);""")

,team_long_name,team_short_name
0,Manchester United,MUN
1,FC Bayern Munich,BMU
2,Real Madrid CF,REA
3,FC Barcelona,BAR


In [6]:
rsq("""SELECT 
    -- Select the country ID and match ID
    country_id, 
    id 
FROM match
-- Filter for matches with 10 or more goals in total
WHERE (home_goal + away_goal) >= 10;""")

,country_id,id
0,1729,3093
1,1729,3369
2,1729,3566
3,7809,9211
4,13274,14224
5,21518,23444
6,21518,24016
7,21518,24114


In [7]:
rsq("""SELECT
    -- Select country name and the count match IDs
    c.name AS country_name,
    COUNT(sub.id) AS matches
FROM country AS c
-- Inner join the subquery onto country
-- Select the country id and match id columns
INNER JOIN (SELECT country_id, id 
            FROM match
            -- Filter the subquery by matches with 10+ goals
            WHERE (home_goal + away_goal) >= 10) AS sub
ON c.id = sub.country_id
GROUP BY country_name;""")

,country_name,matches
0,England,3
1,Germany,1
2,Netherlands,1
3,Spain,3


In [8]:
rsq("""SELECT
    -- Select country, date, home, and away goals from the subquery
    country,
    date,
    home_goal,
    away_goal
FROM
    -- Select country name, date, home_goal, away_goal, and total goals in the subquery
    (SELECT c.name AS country, 
            m.date, 
            m.home_goal, 
            m.away_goal,
           (m.home_goal + m.away_goal) AS total_goals
    FROM match AS m
    LEFT JOIN country AS c
    ON m.country_id = c.id) AS subquery
-- Filter by total goals scored in the main query
WHERE total_goals >= 10;""")

,country,date,home_goal,away_goal
0,England,2011-08-28,8,2.0
1,England,2012-12-29,7,3.0
2,England,2013-05-19,5,5.0
3,Germany,2013-03-30,9,2.0
4,Netherlands,2011-11-06,6,4.0
5,Spain,2013-10-30,7,3.0
6,Spain,2015-04-05,9,1.0
7,Spain,2015-05-23,7,3.0


In [9]:
rsq("""SELECT 
    l.name AS league,
    -- Select and round the league's total goals
    ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Select and round the average total goals
    (SELECT ROUND(AVG(home_goal + away_goal),2) 
     FROM match
     WHERE season = '2013/2014') AS overall_avg
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
-- Filter for the 2013/2014 season
WHERE m.season = '2013/2014'
GROUP BY l.name;""")

,league,avg_goals,overall_avg
0,Germany 1. Bundesliga,3.16,3.19
1,Spain LIGA BBVA,10.00,3.19


In [10]:
rsq("""SELECT 
    -- Select the league name and average goals scored
    l.name AS league,
    ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Subtract the overall average from the league average
    ROUND(AVG(m.home_goal + m.away_goal) - 
          (SELECT AVG(home_goal + away_goal)
           FROM match 
           WHERE season = '2013/2014'),2) AS diff
FROM league AS l
LEFT JOIN match AS m
ON l.country_id = m.country_id
-- Only include 2013/2014 results
WHERE m.season = '2013/2014'
GROUP BY l.name;""")

,league,avg_goals,diff
0,Germany 1. Bundesliga,3.16,-0.03
1,Spain LIGA BBVA,10.00,6.81


In [11]:
rsq("""SELECT 
    -- Select the stage and average goals for each stage
    m.stage,
    ROUND(AVG(m.home_goal + m.away_goal),2) AS avg_goals,
    -- Select the average overall goals for the 2012/2013 season
    ROUND((SELECT AVG(home_goal + away_goal) 
           FROM match 
           WHERE season = '2012/2013'),2) AS overall
FROM match AS m
-- Filter for the 2012/2013 season
WHERE m.season = '2012/2013'
-- Group by stage
GROUP BY m.stage;""")

,stage,avg_goals,overall
0,1,2.26,2.92
1,2,3.40,2.92
2,3,3.42,2.92
3,4,2.67,2.92
4,5,3.67,2.92
5,6,2.33,2.92
6,7,2.44,2.92
7,8,3.22,2.92
8,9,2.89,2.92
9,10,2.95,2.92


In [12]:
rsq("""SELECT 
    -- Select the stage and average goals from s
    s.stage,
    ROUND(s.avg_goals,2) AS avg_goal,
    -- Select the overall average for 2012/2013
    (SELECT AVG(home_goal + away_goal) FROM match WHERE season = '2012/2013') AS overall_avg
FROM 
    -- Select the stage and average goals in 2012/2013 from match
    (SELECT
         stage,
         AVG(home_goal + away_goal) AS avg_goals
     FROM match
     WHERE season = '2012/2013'
     GROUP BY stage) AS s
WHERE 
    -- Filter the main query using the subquery
    s.avg_goals > (SELECT AVG(home_goal + away_goal) 
                   FROM match WHERE season = '2012/2013');""")

,stage,avg_goal,overall_avg
0,2,3.40,2.917778
1,3,3.42,2.917778
2,5,3.67,2.917778
3,8,3.22,2.917778
4,10,2.95,2.917778
5,11,3.37,2.917778
6,12,3.26,2.917778
7,15,3.21,2.917778
8,16,3.00,2.917778
9,17,3.11,2.917778


In [13]:
rsq("""SELECT 
    -- Select the stage and average goals from the subquery
    s.stage,
    ROUND(s.avg_goals,2) AS avg_goals
FROM 
    -- Select the stage and average goals in 2012/2013
    (SELECT
         stage,
         AVG(home_goal + away_goal) AS avg_goals
     FROM match
     WHERE season = '2012/2013'
     GROUP BY stage) AS s
WHERE 
    -- Filter the main query using the subquery
    s.avg_goals > (SELECT AVG(home_goal + away_goal) 
                   FROM match WHERE season = '2012/2013');""")

,stage,avg_goals
0,2,3.40
1,3,3.42
2,5,3.67
3,8,3.22
4,10,2.95
5,11,3.37
6,12,3.26
7,15,3.21
8,16,3.00
9,17,3.11


In [14]:
rsq("""SELECT 
    -- Select the stage and average goals from s
    s.stage,
    ROUND(s.avg_goals,2) AS avg_goal,
    -- Select the overall average for 2012/2013
    (SELECT AVG(home_goal + away_goal) FROM match WHERE season = '2012/2013') AS overall_avg
FROM 
    -- Select the stage and average goals in 2012/2013 from match
    (SELECT
         stage,
         AVG(home_goal + away_goal) AS avg_goals
     FROM match
     WHERE season = '2012/2013'
     GROUP BY stage) AS s
WHERE 
    -- Filter the main query using the subquery
    s.avg_goals > (SELECT AVG(home_goal + away_goal) 
                   FROM match WHERE season = '2012/2013');""")

,stage,avg_goal,overall_avg
0,2,3.40,2.917778
1,3,3.42,2.917778
2,5,3.67,2.917778
3,8,3.22,2.917778
4,10,2.95,2.917778
5,11,3.37,2.917778
6,12,3.26,2.917778
7,15,3.21,2.917778
8,16,3.00,2.917778
9,17,3.11,2.917778
